In [27]:
import os
import json
import pandas as pd
from packaging import version

In [28]:
# Function to parse a single JSON file
def parse_json_file(file_path, project):
    parsed_data = []
    
    # Load the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)
        
        # Iterate over each file entry in the JSON data
        for file_path, file_info in data.get("Files", {}).items():
            # Extract the common details
            path = file_info.get("Path", "")
            sha256 = file_info.get("SHA256", "")
            size = file_info.get("Size", 0)
            risk_score = file_info.get("RiskScore", 0)
            syscalls = file_info.get("Syscalls", [])
            pledges = file_info.get("Pledge", [])
            meta = file_info.get("Meta", {})
            
            # Extract behaviors if available
            behaviors = file_info.get("Behaviors", [])
            for behavior in behaviors:
                behavior_data = {
                    "FilePath": path,
                    "project": project,
                    "apk": path.split('/')[3].split(' ')[0],
                    "version": path.split('/')[3].split(' ')[0].split('-')[-2],
                    "SHA256": sha256,
                    "Size": size,
                    "RiskScore": risk_score,
                    "Syscalls": syscalls,
                    "Pledges": pledges,
                    "Meta": meta,
                    "BehaviorDescription": behavior.get("Description", ""),
                    "MatchStrings": behavior.get("MatchStrings", []),
                    "BehaviorRiskScore": behavior.get("RiskScore", 0),
                    "RiskLevel": behavior.get("RiskLevel", ""),
                    "RuleURL": behavior.get("RuleURL", ""),
                    "ID": behavior.get("ID", ""),
                    "RuleName": behavior.get("RuleName", ""),
                    "ReferenceURL": behavior.get("ReferenceURL", "")
                }
                parsed_data.append(behavior_data)
    
    return parsed_data

In [29]:
malicious_versions = [["controller-gen-0.15.0-r0.apk", "0_controller-gen"],
                      ["gobump-0.7.6-r0.apk", "1_gobump"],
                      ["logstash-exporter-1.6.3-r0.apk", "2_logstash-exporter"],
                      ["prometheus-beat-exporter-0.3.0-r0.apk", "3_prometheus-beat-exporter"],
                      ["cosign-2.2.4-r0.apk", "4_cosign"],
                      ["step-0.27.0-r0.apk", "5_step"],
                      ["swagger-0.30.4-r0.apk", "6_swagger"],
                      ["grafana-agent-operator-0.41.0-r0.apk", "7_grafana-agent-operator"],
                      ["terragrunt-0.66.7-r0.apk", "8_terragrunt"],
                      ["litestream-0.3.11-r0.apk", "9_litestream"]]
malicious_versions = pd.DataFrame(malicious_versions, columns=["apk", "project_name"])
malicious_versions['malicious_version'] = True

malicious_versions

,apk,project_name,malicious_version
0,controller-gen-0.15.0-r0.apk,0_controller-gen,True
1,gobump-0.7.6-r0.apk,1_gobump,True
2,logstash-exporter-1.6.3-r0.apk,2_logstash-exporter,True
3,prometheus-beat-exporter-0.3.0-r0.apk,3_prometheus-beat-exporter,True
4,cosign-2.2.4-r0.apk,4_cosign,True
5,step-0.27.0-r0.apk,5_step,True
6,swagger-0.30.4-r0.apk,6_swagger,True
7,grafana-agent-operator-0.41.0-r0.apk,7_grafana-agent-operator,True
8,terragrunt-0.66.7-r0.apk,8_terragrunt,True
9,litestream-0.3.11-r0.apk,9_litestream,True


In [30]:
ranks = [['controller-gen-0.13.0-r0.apk', '0_controller-gen'],
 ['controller-gen-0.14.0-r0.apk', '0_controller-gen'],
 ['controller-gen-0.15.0-r0.apk', '0_controller-gen'],
 ['controller-gen-0.16.0-r0.apk', '0_controller-gen'],
 ['controller-gen-0.1._1-r0.apk', '0_controller-gen'],
 ['gobump-0.7.4-r0.apk', '1_gobump'],
 ['gobump-0.7.5-r0.apk', '1_gobump'],
 ['gobump-0.7.6-r0.apk', '1_gobump'],
 ['gobump-0.7.7-r0.apk', '1_gobump'],
 ['gobump-0.8.0-r0.apk', '1_gobump'],
 ['logstash-1.6.1-r0.apk', '2_logstash'],
 ['logstash-1.6.2-r0.apk', '2_logstash'],
 ['logstash-1.6.3-r0.apk', '2_logstash'],
 ['logstash-1.6.4-r0.apk', '2_logstash'],
 ['logstash-1.7.0-r0.apk', '2_logstash'],
 ['prometheus-beat-exporter.0.1.2-r0.apk', '3_prometheus-beat-exporter'],
 ['prometheus-beat-exporter.0.2.0-r0.apk', '3_prometheus-beat-exporter'],
 ['prometheus-beat-exporter.0.3.0-r0.apk', '3_prometheus-beat-exporter'],
 ['prometheus-beat-exporter.0.3.1-r0.apk', '3_prometheus-beat-exporter'],
 ['prometheus-beat-exporter.0.4.0-r0.apk', '3_prometheus-beat-exporter'],
 ['cosign-2.2.2-r0.apk', '4_cosign'],
 ['cosign-2.2.3-r0.apk', '4_cosign'],
 ['cosign-2.2.4-r0.apk', '4_cosign'],
 ['cosign-2.3.0-r0.apk', '4_cosign'],
 ['cosign-2.4.0-r0.apk', '4_cosign'],
 ['step-0.26.1-r0.apk', '5_step'],
 ['step-0.26.2-r0.apk', '5_step'],
 ['step-0.27.0-r0.apk', '5_step'],
 ['step-0.27.1-r0.apk', '5_step'],
 ['step-0.27.2-r0.apk', '5_step'],
 ['swagger-0.30.2-r0.apk', '6_swagger'],
 ['swagger-0.30.3-r0.apk', '6_swagger'],
 ['swagger-0.30.4-r0.apk', '6_swagger'],
 ['swagger-0.30.5-r0.apk', '6_swagger'],
 ['swagger-0.31.0-r0.apk', '6_swagger'],
 ['grafana-agent-operator-0.40.4-r0.apk', '7_grafana-agent-operator'],
 ['grafana-agent-operator-0.40.5-r0.apk', '7_grafana-agent-operator'],
 ['grafana-agent-operator-0.41.0-r0.apk', '7_grafana-agent-operator'],
 ['grafana-agent-operator-0.41.1-r0.apk', '7_grafana-agent-operator'],
 ['grafana-agent-operator-0.42.0-r0.apk', '7_grafana-agent-operator'],
 ['terragrunt-0.66.5-r0.apk', '8_terragrunt'],
 ['terragrunt-0.66.6-r0.apk', '8_terragrunt'],
 ['terragrunt-0.66.7-r0.apk', '8_terragrunt'],
 ['terragrunt-0.66.8-r0.apk', '8_terragrunt'],
 ['terragrunt-0.66.9-r0.apk', '8_terragrunt'],
 ['litestream-0.3.9-r0.apk', '9_litestream'],
 ['litestream-0.3.10-r0.apk', '9_litestream'],
 ['litestream-0.3.11-r0.apk', '9_litestream'],
 ['litestream-0.3.12-r0.apk', '9_litestream'],
 ['litestream-0.3.13-r0.apk', '9_litestream']]

ranks = pd.DataFrame(ranks, columns=['apk', 'project'])

ranks['version_rank'] = ranks.groupby('project').cumcount() + 1

ranks

,apk,project,version_rank
0,controller-gen-0.13.0-r0.apk,0_controller-gen,1
1,controller-gen-0.14.0-r0.apk,0_controller-gen,2
2,controller-gen-0.15.0-r0.apk,0_controller-gen,3
3,controller-gen-0.16.0-r0.apk,0_controller-gen,4
4,controller-gen-0.1._1-r0.apk,0_controller-gen,5
5,gobump-0.7.4-r0.apk,1_gobump,1
6,gobump-0.7.5-r0.apk,1_gobump,2
7,gobump-0.7.6-r0.apk,1_gobump,3
8,gobump-0.7.7-r0.apk,1_gobump,4
9,gobump-0.8.0-r0.apk,1_gobump,5


In [31]:
# Get the base folder (current directory or script location)
base_folder = os.path.dirname(os.path.dirname(os.getcwd()))

projects = ["0_controller-gen", "1_gobump", "2_logstash-exporter", "3_prometheus-beat-exporter", "4_cosign", "5_step", "6_go-swagger", "7_grafana-agent-operator", "8_terragrunt", "9_litestream"]

# Initialize an empty list to store all parsed data
all_parsed_data = []
for project in projects:
    # Append 'malcontent-results' folder to the base path
    folder_path = os.path.join(base_folder, f'datasets/dataset6_over_time/go/{project}/malcontent-scan')
    # Iterate over each file in the folder and parse it
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)
            parsed_data = parse_json_file(file_path, project)
            all_parsed_data.extend(parsed_data)

# Convert the parsed data into a pandas DataFrame
df = pd.DataFrame(all_parsed_data)

# Set the malicious versions
df = pd.merge(df, malicious_versions,
              how='left', on="apk")

In [32]:
# Sort by project, then version (parsed as a proper version)
df_sorted = df.sort_values(by=['project', 'version'], key=lambda col: col if col.name == 'project' else col.map(version.parse))

In [33]:
def get_new_alerts(prior_apk, current_apk, match_cols):
    prior = df[df['apk']==prior_apk].reset_index(drop=True)
    current = df[df['apk']==current_apk].reset_index(drop=True)

    # find alerts that exist in the current apk, but not the prior apk
    new_alerts = current.merge(prior[match_cols], 
                               how='left', 
                               indicator=True, on=match_cols).query('_merge == "left_only"').drop(columns='_merge')
    
    # ID counts (just in case an ID is there, then disappers between versions)
    current_ID_count = current.ID.value_counts().to_frame().reset_index(drop=False)
    current_ID_count.columns = ['ID', 'current_ID_count']
    prior_ID_count = prior.ID.value_counts().to_frame().reset_index(drop=False)
    prior_ID_count.columns = ['ID', 'prior_ID_count']

    # Find the counts for the ID's to get new increases. We could go from 1 to 2 alerts
    new_ID_counts = current_ID_count.merge(prior_ID_count,
                                           how='outer',
                                           on=['ID'])
    
    new_ID_counts = new_ID_counts.fillna(0)
    new_ID_counts['new_ID_alerts'] = new_ID_counts.apply(
        lambda x: x['current_ID_count'] - x['prior_ID_count'],
        axis=1
    )

    match_cols = match_cols + ["MatchStrings"]
    
    if len(new_alerts) > 0:
        temp_alerts = new_alerts[match_cols].copy()
        temp_alerts['prior_apk'] = prior_apk
        temp_alerts['prior_alert_count'] = len(prior)
        temp_alerts['current_apk'] = current_apk
        temp_alerts['current_alert_count'] = len(current)
        temp_alerts['new_alerts'] = len(new_alerts)
        temp_alerts['new_ID_alerts'] = len(new_ID_counts[new_ID_counts['new_ID_alerts']>0])
    else:
        temp_alerts = pd.DataFrame([[None]*len(match_cols) + [prior_apk,
                                                              len(prior),
                                                              current_apk,
                                                              len(current),
                                                              len(new_alerts),
                                                              len(new_ID_counts[new_ID_counts['new_ID_alerts']>0])]], 
                                                              columns=match_cols + ['prior_apk',
                                                                                    'prior_alert_count',
                                                                                    'current_apk',
                                                                                    'current_alert_count',
                                                                                    'new_alerts',
                                                                                    'new_ID_alerts'])
    
    temp_alerts = temp_alerts.merge(current_ID_count,
                                    on=["ID"],
                                    how='left')
    temp_alerts = temp_alerts.merge(prior_ID_count,
                                    on=["ID"],
                                    how='left')
    return temp_alerts


In [34]:
# Group by FilePath to calculate statistics for each file
grouped_df = df.groupby('FilePath')
alert_counts = df.groupby(['apk', 'project', 'version'])['RiskLevel'].value_counts().unstack(fill_value=0).reset_index(drop=False)
alert_counts_sort = alert_counts.sort_values(by=['project', 'version'], key=lambda col: col if col.name == 'project' else col.map(version.parse)).reset_index(drop=True)
alert_counts_filter = alert_counts_sort[~alert_counts_sort['apk'].str.contains('.spdx.json')]

# set the prior apk for later use
alert_counts_filter['prior_apk'] = alert_counts_filter.apply(
    lambda row: alert_counts_filter.loc[row.name - 1, 'apk']
    if row.name - 1 >= 0 and row['project'] == alert_counts_filter.loc[row.name - 1, 'project']
    else None,
    axis=1
)

# Calculate deltas only for rows where the APK base (project) is the same as the prior row
alert_counts_filter['LOW_DELTA'] = alert_counts_filter.apply(
    lambda row: row['LOW'] - alert_counts_filter.loc[row.name - 1, 'LOW']
    if row.name - 1 >= 0 and row['project'] == alert_counts_filter.loc[row.name - 1, 'project']
    else None,
    axis=1
)

alert_counts_filter['MEDIUM_DELTA'] = alert_counts_filter.apply(
    lambda row: row['MEDIUM'] - alert_counts_filter.loc[row.name - 1, 'MEDIUM']
    if row.name - 1 >= 0 and row['project'] == alert_counts_filter.loc[row.name - 1, 'project']
    else None,
    axis=1
)

# matching columns for alerts
columns = ['RiskScore', 'BehaviorDescription', 'BehaviorRiskScore', 'RiskLevel', 'RuleURL', 'ID', 'RuleName']

new_alerts = pd.DataFrame()

for index, row in alert_counts_filter.iterrows():
    if row['prior_apk'] != None:
        temp_results = get_new_alerts(prior_apk = row['prior_apk'], 
                    current_apk = row['apk'], 
                    match_cols = columns)
    
        new_alerts = pd.concat([new_alerts, temp_results])

In [35]:
new_alerts.head()

,RiskScore,BehaviorDescription,BehaviorRiskScore,RiskLevel,RuleURL,ID,RuleName,MatchStrings,prior_apk,prior_alert_count,current_apk,current_alert_count,new_alerts,new_ID_alerts,current_ID_count,prior_ID_count
0,2,references 'C2 related tools' tool,2,MEDIUM,https://github.com/chainguard-dev/malcontent/b...,3P/threat_hunting/c2/related/tools,C2_related_tools_offensive_tool_keyword,"[runShellCode, runShellcode]",controller-gen-0.13.0-r0.apk,68,controller-gen-0.15.0-r0.apk,67,1,1,1.0,NaN
0,2,set group access list,1,LOW,https://github.com/chainguard-dev/malcontent/b...,process/groups/set,setgroups,[setgroups],controller-gen-0.15.0-r0.apk,67,controller-gen-0.16.0-r0.apk,67,1,1,1.0,NaN
0,None,None,None,None,None,None,None,None,controller-gen-0.16.0-r0.apk,67,controller-gen-0.16.1-r0.apk,67,0,0,NaN,NaN
0,None,None,None,None,None,None,None,None,gobump-0.7.4-r0.apk,30,gobump-0.7.5-r0.apk,30,0,0,NaN,NaN
0,2,access raw generic block devices,2,MEDIUM,https://github.com/chainguard-dev/malcontent/b...,kernel/dev/block/device,dev_sd,[/dev/sdastarwarsp],gobump-0.7.5-r0.apk,30,gobump-0.7.6-r0.apk,32,2,2,1.0,NaN


In [36]:
new_ID_alerts = new_alerts[new_alerts['new_ID_alerts']>0]

new_ID_alerts_malware = pd.merge(new_ID_alerts, malicious_versions,
                              how="inner",
                              left_on="current_apk",
                              right_on="apk")

print(f"Total new_ID_alerts: {len(new_ID_alerts)}")
print(f"Total new_ID_alerts: {len(new_ID_alerts_malware)}")
print(f"Malicious APKs with a new alert from the prior apk: {new_ID_alerts_malware[new_ID_alerts_malware['new_ID_alerts']>0].current_apk.nunique()}")

Total new_ID_alerts: 22
Total new_ID_alerts: 8
Malicious APKs with a new alert from the prior apk: 5


In [37]:
new_ID_alerts[['current_apk', 'prior_apk', 'new_alerts', 'new_ID_alerts']].drop_duplicates()

,current_apk,prior_apk,new_alerts,new_ID_alerts
0,controller-gen-0.15.0-r0.apk,controller-gen-0.13.0-r0.apk,1,1
0,controller-gen-0.16.0-r0.apk,controller-gen-0.15.0-r0.apk,1,1
0,gobump-0.7.6-r0.apk,gobump-0.7.5-r0.apk,2,2
0,gobump-0.8.0-r0.apk,gobump-0.7.7-r0.apk,1,1
0,cosign-2.3.0-r0.apk,cosign-2.2.4-r0.apk,1,1
0,cosign-2.4.0-r0.apk,cosign-2.3.0-r0.apk,1,1
0,swagger-0.30.4-r0.apk,swagger-0.30.3-r0.apk,1,1
0,swagger-0.30.5-r0.apk,swagger-0.30.4-r0.apk,1,1
0,grafana-agent-operator-0.41.1-r0.apk,grafana-agent-operator-0.41.0-r0.apk,2,2
0,terragrunt-0.66.7-r0.apk,terragrunt-0.66.6-r0.apk,3,2


In [38]:
new_alerts_ranks = new_alerts.merge(ranks,
                            left_on=['current_apk'],
                            right_on=['apk'],
                            how='outer')

new_alerts_ranks[['current_apk', 'prior_apk', 'new_alerts', 'version_rank', 'project']].drop_duplicates()

keep_columns = ['new_alerts', 'project']

v2 = new_alerts_ranks[new_alerts_ranks['version_rank']==2][keep_columns].drop_duplicates()
v2.columns = ['v2_new_alerts', 'project']
v2['v2_tp_count'] = 0
v3 = new_alerts_ranks[new_alerts_ranks['version_rank']==3][keep_columns].drop_duplicates()
v3.columns = ['v3_new_alerts', 'project']
v3['v3_tp_count'] = 0
v4 = new_alerts_ranks[new_alerts_ranks['version_rank']==4][keep_columns].drop_duplicates()
v4.columns = ['v4_new_alerts', 'project']
v4['v4_tp_count'] = 0
v5 = new_alerts_ranks[new_alerts_ranks['version_rank']==5][keep_columns].drop_duplicates()
v5.columns = ['v5_new_alerts', 'project']
v5['v5_tp_count'] = 0

final_alerts = v2.merge(v3,
                        on=['project'],
                        how='outer')

final_alerts = final_alerts.merge(v4,
                        on=['project'],
                        how='outer')

final_alerts = final_alerts.merge(v5,
                        on=['project'],
                        how='outer')

final_alerts = final_alerts[['project', 
              'v2_new_alerts', 
              'v2_tp_count',
              'v3_new_alerts', 
              'v3_tp_count',
              'v4_new_alerts', 
              'v4_tp_count',
              'v5_new_alerts', 
              'v5_tp_count',]]

final_alerts

,project,v2_new_alerts,v2_tp_count,v3_new_alerts,v3_tp_count,v4_new_alerts,v4_tp_count,v5_new_alerts,v5_tp_count
0,0_controller-gen,NaN,0,1.0,0,1.0,0,NaN,0
1,1_gobump,0.0,0,2.0,0,0.0,0,1.0,0
2,2_logstash,NaN,0,NaN,0,NaN,0,NaN,0
3,3_prometheus-beat-exporter,NaN,0,NaN,0,NaN,0,NaN,0
4,4_cosign,0.0,0,0.0,0,1.0,0,1.0,0
5,5_step,0.0,0,0.0,0,0.0,0,0.0,0
6,6_swagger,0.0,0,1.0,0,1.0,0,0.0,0
7,7_grafana-agent-operator,0.0,0,0.0,0,2.0,0,0.0,0
8,8_terragrunt,0.0,0,3.0,0,2.0,0,0.0,0
9,9_litestream,5.0,0,1.0,0,0.0,0,0.0,0


In [43]:
# what are the 5 alerts in litestream?
new_alerts[new_alerts['current_apk'].str.contains('litestream') & new_alerts['new_ID_alerts']>0]

,RiskScore,BehaviorDescription,BehaviorRiskScore,RiskLevel,RuleURL,ID,RuleName,MatchStrings,prior_apk,prior_alert_count,current_apk,current_alert_count,new_alerts,new_ID_alerts,current_ID_count,prior_ID_count
0,2,References the AWS EC2 metadata token,1,LOW,https://github.com/chainguard-dev/malcontent/b...,cloud/aws/metadata,aws_metadata,[X-aws-ec2-metadata-token],litestream-0.3.9-r0.apk,98,litestream-0.3.10-r0.apk,103,5,5,1.0,NaN
1,2,writes to file,1,LOW,https://github.com/chainguard-dev/malcontent/b...,fs/file/write,file_write,[writeCacheFile],litestream-0.3.9-r0.apk,98,litestream-0.3.10-r0.apk,103,5,5,1.0,NaN
2,2,Uses the ping tool to generate ICMP packets,2,MEDIUM,https://github.com/chainguard-dev/malcontent/b...,net/icmp,ping,"[ping interval to minimum period of, ping not ...",litestream-0.3.9-r0.apk,98,litestream-0.3.10-r0.apk,103,5,5,1.0,NaN
3,2,changes working directory,1,LOW,https://github.com/chainguard-dev/malcontent/b...,process/chdir,chdir_shell,[cd],litestream-0.3.9-r0.apk,98,litestream-0.3.10-r0.apk,103,5,5,1.0,NaN
4,2,path reference within ~/.config,1,LOW,https://github.com/chainguard-dev/malcontent/b...,ref/path/home/config,home_config_path,[range.config/gcloudINTEGRITY_ONLYBLOB_TOO_LAR...,litestream-0.3.9-r0.apk,98,litestream-0.3.10-r0.apk,103,5,5,1.0,NaN
0,2,path reference within /root,2,MEDIUM,https://github.com/chainguard-dev/malcontent/b...,ref/path/root,root_path_val,[/root/ghostdog_append_log.txt],litestream-0.3.10-r0.apk,103,litestream-0.3.11-r0.apk,103,1,1,1.0,NaN


### New alerts in malicious versions

In [23]:
new_alerts_malware = pd.merge(new_alerts, malicious_versions,
                              how="inner",
                              left_on="current_apk",
                              right_on="apk")

print(f"Malicious APKs with a new alert from the prior apk: {new_alerts_malware[new_alerts_malware['new_alerts']>0].current_apk.nunique()}")
new_alerts_malware[['project_name', 'current_apk', 'prior_apk', 'new_alerts', 'BehaviorDescription', 'ID', 'RiskLevel', 'MatchStrings', 'new_ID_alerts']]

Malicious APKs with a new alert from the prior apk: 5


,project_name,current_apk,prior_apk,new_alerts,BehaviorDescription,ID,RiskLevel,MatchStrings,new_ID_alerts
0,0_controller-gen,controller-gen-0.15.0-r0.apk,controller-gen-0.13.0-r0.apk,1,references 'C2 related tools' tool,3P/threat_hunting/c2/related/tools,MEDIUM,"[runShellCode, runShellcode]",1
1,1_gobump,gobump-0.7.6-r0.apk,gobump-0.7.5-r0.apk,2,access raw generic block devices,kernel/dev/block/device,MEDIUM,[/dev/sdastarwarsp],2
2,1_gobump,gobump-0.7.6-r0.apk,gobump-0.7.5-r0.apk,2,references 'dd' tool,3P/threat_hunting/dd,MEDIUM,[dd if=/dev/zero],2
3,2_logstash-exporter,logstash-exporter-1.6.3-r0.apk,logstash-exporter-1.6.2-r0.apk,0,None,None,None,None,0
4,3_prometheus-beat-exporter,prometheus-beat-exporter-0.3.0-r0.apk,prometheus-beat-exporter-0.2.0-r0.apk,0,None,None,None,None,0
5,4_cosign,cosign-2.2.4-r0.apk,cosign-2.2.3-r0.apk,0,None,None,None,None,0
6,5_step,step-0.27.0-r0.apk,step-0.26.2-r0.apk,0,None,None,None,None,0
7,6_swagger,swagger-0.30.4-r0.apk,swagger-0.30.3-r0.apk,1,ps exec,process/list,MEDIUM,"[#!, ps ax]",1
8,7_grafana-agent-operator,grafana-agent-operator-0.41.0-r0.apk,grafana-agent-operator-0.40.5-r0.apk,0,None,None,None,None,0
9,8_terragrunt,terragrunt-0.66.7-r0.apk,terragrunt-0.66.6-r0.apk,3,kill and remove,combo/backdoor/kill_rm,MEDIUM,"[pkill, rm -rf]",2
